In [1]:
from collections import defaultdict

import folium
import geopandas as gpd
import matplotlib as mpl
import matplotlib.colors as mcolors
import numpy as np
import pyproj
import rioxarray  # noqa: F401
import shapely
import xarray as xr
from darts_acquisition.utils.grid import MajorTomGrid

# Major Tom Grid visualizations

This notebook covers the Major Tom Grid for specific examples (Banks Island).

In [ ]:
# Define the grid
grid = MajorTomGrid(10)
len(grid)  # Total cells in the grid

In [ ]:
# Define the Banks Island shape (in EPSG:4326 - Lat/Lon)
geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [
                        [-125.52021773630446, 74.43951408973027],
                        [-124.86584691986315, 73.56494956988854],
                        [-126.34066074572485, 71.92089501178256],
                        [-124.1659286262622, 71.41732439527635],
                        [-123.12804207859187, 70.9208989281801],
                        [-120.53032198921977, 71.3707480482829],
                        [-119.70236171777628, 71.93986525345568],
                        [-118.44459474592787, 72.68254360652071],
                        [-116.8649170906869, 73.06781111551533],
                        [-114.65553106718167, 73.41365769135606],
                        [-117.02781329772819, 74.41235509069708],
                        [-119.51816697734262, 74.4307727183745],
                        [-121.11534855024786, 74.70854802770796],
                        [-125.52021773630446, 74.43951408973027],
                    ]
                ],
                "type": "Polygon",
            },
        }
    ],
}
shape = gpd.GeoDataFrame.from_features(geojson_data, crs="EPSG:4326")["geometry"][0]
shape

In [ ]:
# Export the grid to a geodataframe for later visualization
gdf = grid.to_geodataframe(bounds=shape)
gdf

## Simulate tiles in their respective UTM Zones (m)

In [ ]:
# Use Sentinel 2 resolution and tile-size from the major-tom paper (https://arxiv.org/html/2402.12095v2)
size = 1068  # px
resolution = 10  # m / px

# We need to create a dataframe for each UTM zone, since all rows of a dataframe must have the same CRS
cell_bounds = defaultdict(list)
for i, point, d, row_idx, col_idx, row_direction, col_direction, lat, lon, utm_zone in gdf.itertuples():
    # Convert the lat / lon to their respective UTM-zone
    # and define the coordinate grid of the tile based on size and resolution
    transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{utm_zone}", always_xy=True)
    x_start, y_start = transformer.transform(point.x, point.y)
    x_coords = np.linspace(x_start, x_start + (size * resolution), size)
    y_coords = np.linspace(y_start, y_start + (size * resolution), size)
    # Create a virtual tile to use rioxarray to get the bounds and convert them into a shapely geometry
    # This step could also be done by just calculating the bounds manually, however I want to show
    # how to use rioxarray to do this
    tile = xr.DataArray(0.0, coords=[("x", x_coords), ("y", y_coords)])
    tile.rio.set_spatial_dims("x", "y", inplace=True)
    tile.rio.write_crs(f"EPSG:{utm_zone}", inplace=True)
    bound_box = shapely.geometry.box(*tile.rio.bounds())
    # Store the bounds and other information in a dictionary for later conversion to a geodataframe
    cell_bounds[utm_zone].append(
        {
            "geometry": bound_box,
            "row_idx": row_idx,
            "col_idx": col_idx,
            "row_direction": row_direction,
            "col_direction": col_direction,
            "lat": lat,
            "lon": lon,
            "utm_zone": utm_zone,
        }
    )
cell_bounds = {utm_zone: gpd.GeoDataFrame(cell_bounds[utm_zone], crs=f"EPSG:{utm_zone}") for utm_zone in cell_bounds}

# Map the UTM zones to a color
cmap = mpl.colormaps["viridis"].resampled(len(cell_bounds))
utm_cmappings = {
    utm_zone: mcolors.to_hex(cmap(i / len(cell_bounds))) for i, (utm_zone, _) in enumerate(cell_bounds.items())
}
# Visualize the grid
m = folium.Map(tiles="cartodb darkmatter")
# Add the Banks Island shape
folium.GeoJson(shape.__geo_interface__, name="Banks Island").add_to(m, 0)
# Add the grid boundaries
for utm_zone, gdf_boundaries in cell_bounds.items():
    gdf_boundaries.explore(m=m, color=utm_cmappings[utm_zone], name=f"Grid Boundaries {utm_zone}")
# Add the grid points
gdf_colors = gdf.utm_zone.map(utm_cmappings)
gdf.explore(color=gdf_colors, m=m, marker_kwds={"radius": 3}, name="Grid Points")
# Add UI controls
folium.FitOverlays().add_to(m)
folium.LayerControl().add_to(m)
m

## Simulate reprojection to EPSG:4326 (lat/lon)

Now I want to show the offset of the coordinates when converting the final bounds to lat lon.

This should reflect a workflow were each tiles are generated and processed in their respective UTM coordinate system and
later combined.

In the visualisation it should be visible that the points and tile-starts are slightly shifted

In [ ]:
# Now we only create a single dataframe, since they will all share the same CRS (EPSG:4326)
cell_bounds = []
for i, point, d, row_idx, col_idx, row_direction, col_direction, lat, lon, utm_zone in gdf.itertuples():
    # Convert the lat / lon to their respective UTM-zone
    # and define the coordinate grid of the tile based on size and resolution
    transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{utm_zone}", always_xy=True)
    x_start, y_start = transformer.transform(point.x, point.y)
    x_coords = np.linspace(x_start, x_start + (size * resolution), size)
    y_coords = np.linspace(y_start, y_start + (size * resolution), size)
    # Create a virtual tile to use rioxarray to get the bounds and convert them into a shapely geometry
    # This step could also be done by just calculating the bounds manually, however I want to show
    # how to use rioxarray to do this
    tile = xr.DataArray(0.0, coords=[("x", x_coords), ("y", y_coords)])
    tile.rio.set_spatial_dims("x", "y", inplace=True)
    tile.rio.write_crs(f"EPSG:{utm_zone}", inplace=True)
    # Transform the bounds back to EPSG:4326
    bound_box = shapely.geometry.box(*tile.rio.transform_bounds("EPSG:4326"))
    # Store the bounds and other information in a dictionary for later conversion to a geodataframe
    cell_bounds.append(
        {
            "geometry": bound_box,
            "row_idx": row_idx,
            "col_idx": col_idx,
            "row_direction": row_direction,
            "col_direction": col_direction,
            "lat": lat,
            "lon": lon,
            "utm_zone": utm_zone,
        }
    )
cell_bounds = gpd.GeoDataFrame(cell_bounds, crs="EPSG:4326")

# Visualize the grid
m = folium.Map(tiles="cartodb darkmatter")
# Add the Banks Island shape
folium.GeoJson(shape.__geo_interface__, name="Banks Island").add_to(m, 0)
# Add the grid boundaries
cell_bounds.explore(color=gdf_colors, m=m, name="Grid Boundaries")
# Add the grid points
gdf_colors = gdf.utm_zone.map(utm_cmappings)
gdf.explore(color=gdf_colors, m=m, marker_kwds={"radius": 3}, name="Grid Points")
# Add UI controls
folium.FitOverlays().add_to(m)
folium.LayerControl().add_to(m)
m